# Missing Value

데이터를 수집해서 처리하다보면 결측치를 만나게 됩니다. 결측치가 포함된 모델은 성능에 안 좋은 영향을 미칠 수 있기 때문에 전처리시에 처리하게 됩니다. 보통은 프로젝트내 도메인 전문가의 조언대로 처리되지만, 없을 경우는 결측치의 원인과 종류에 따라 그 처리 방법을 각기 달리 해서 처리합니다. 

## Missing Value의 종류
Missing Completely at Random (MCAR)
* 완전하게 랜덤으로 결측치가 나타나는 경우입니다.
* 완전하게 랜덤으로 나타난다면, 데이터가 큰 경우 랜덤 샘플링을 통해 완전한 데이터를 만들수 있게 됩니다.


Missing at Random (MAR)
* 결측치가 특정 변수와 관련되어 일어나지만 그 변수의 값과는 관계가 없는 경우입니다.
* 예: 결측치가 발견되었는데, 데이터 수집 과정에서 설문 응답자가 다음 페이지가 있는지 모르고 응답을 종료한 경우 


Missing not at Random (MNAR)
* 결측치의 값과 결측 이유가 관련이 있는 경우입니다. 
* 예: 결측치가 발견되었는데, 데이터 수집 과정에서 설문 응답자가 해당 수집 변수에 대해 응답을 꺼려하여 응답하지 않은 경우

## Missing Value 처리 기준

'Multivariate Data Analysis' Pearson Education 책에서는 아래 방법을 추천하고 있습니다.

* 결측치 비율 10% 이하: 어떤 Imputation 방법도 상관 없음
* 결측치 비율 10% ~ 20%: MCAR일 경우 Replace, Regression 방법 추천, MAR 일 경우 모델기반 방법 추천
* 결측치 비율 20% 이상: MCAR일 경우 Regression 방법 추천, MAR 일 경우 모델기반 방법 추천

추가적으로 결측치 비율 10% 이하이고, 데이터가 빅데이터인 경우는 Deletion 방법도 고려해 볼 수 있습니다.

In [54]:
import pandas as pd 
import numpy as np

df = pd.DataFrame(np.random.randn(5, 3), index=range(5), columns=['one', 'two', 'three'])
df.iloc[2:, 1] = np.nan
df.iat[-1, 0] = np.nan
print(df)

# 결측치 비율 구하기
df_missing = pd.DataFrame(df.isnull().sum(), columns=['missing'])
df_missing["percent"] = df_missing['missing'] / float(len(df.index)) * 100
print(df_missing)

        one       two     three
0 -1.951405 -0.202758  0.803260
1  0.196079 -1.185378  0.043634
2  0.941370       NaN -2.141942
3  0.093330       NaN  0.784660
4       NaN       NaN  1.177910
       missing  percent
one          1     20.0
two          3     60.0
three        0      0.0


## Missing Value 처리 방법

### Deletion
#### Listwise Deletion
결측치가 발생된 행 전체 삭제

In [55]:
import pandas as pd 
import numpy as np

df = pd.DataFrame(np.random.randn(5, 3), index=range(5), columns=['one', 'two', 'three'])
df.iloc[2:, 1] = np.nan
df.iat[-1, 0] = np.nan
df.iat[-1, 2] = np.nan
print(df)

print(df.dropna(how='any'))

        one       two     three
0 -0.757771 -0.487931 -0.408137
1 -1.836704  0.027993 -0.627672
2  0.714692       NaN  0.220564
3  0.210239       NaN -1.240897
4       NaN       NaN       NaN
        one       two     three
0 -0.757771 -0.487931 -0.408137
1 -1.836704  0.027993 -0.627672


#### Pairwise Deletion
통계에 따라 선택된 결측치만 삭제, 통계적 지식이 필요하고, 주의하지 않으면 오류를 포함할수 있어 잘 사용되지 않음

#### Deleting Columns
결측치가 발생된 열 전체 삭제, 해당 열이 종속 변수와 상관 관계가 없는 경우만 사용

In [57]:
import pandas as pd 
import numpy as np

df = pd.DataFrame(np.random.randn(5, 3), index=range(5), columns=['one', 'two', 'three'])
df.iloc[:, 1] = np.nan
df.iat[-1, 0] = np.nan
df.iat[-1, 2] = np.nan
print(df)

print(df.dropna(how='all', axis=1))

        one  two     three
0  2.960500  NaN  0.996960
1 -0.672128  NaN  0.985775
2 -1.338470  NaN -0.010889
3  0.237134  NaN -0.497537
4       NaN  NaN       NaN
        one     three
0  2.960500  0.996960
1 -0.672128  0.985775
2 -1.338470 -0.010889
3  0.237134 -0.497537
4       NaN       NaN


### Imputation
#### Record Data
##### Continuous Data

In [116]:
import pandas as pd 
import numpy as np

df = pd.DataFrame(np.random.randn(5, 3), index=range(5), columns=['one', 'two', 'three'])
df.iat[-1, 0] = np.nan
df.iat[-1, 1] = np.nan
df.iloc[2:, 2] = np.nan
print(df)

# Mean(평균값)으로 채우기
mean = df['one'].mean()
print(mean)
df['one'].fillna(mean, inplace=True)
print(df)

# Median(중앙값)으로 채우기
median = df['two'].median()
print(median)
df['two'].fillna(median, inplace=True)
print(df)

# Constant(상수값)으로 채우기
df['three'].fillna(0, inplace=True)
print(df)

        one       two     three
0  0.342316  2.135612 -0.510639
1 -0.890561  0.067181 -0.208658
2  0.379179 -0.456647       NaN
3 -1.870534 -0.364213       NaN
4       NaN       NaN       NaN
-0.5099002655528244
        one       two     three
0  0.342316  2.135612 -0.510639
1 -0.890561  0.067181 -0.208658
2  0.379179 -0.456647       NaN
3 -1.870534 -0.364213       NaN
4 -0.509900       NaN       NaN
-0.14851643836664008
        one       two     three
0  0.342316  2.135612 -0.510639
1 -0.890561  0.067181 -0.208658
2  0.379179 -0.456647       NaN
3 -1.870534 -0.364213       NaN
4 -0.509900 -0.148516       NaN
        one       two     three
0  0.342316  2.135612 -0.510639
1 -0.890561  0.067181 -0.208658
2  0.379179 -0.456647  0.000000
3 -1.870534 -0.364213  0.000000
4 -0.509900 -0.148516  0.000000


In [120]:
import pandas as pd 
import numpy as np

df = pd.DataFrame(np.random.randn(5, 3), index=range(5), columns=['one', 'two', 'three'])
df.iat[-1, 0] = np.nan
df.iat[-1, 1] = np.nan
df.iloc[2:, 2] = np.nan
print(df)

# 모델로 채우기 - MICE
# Multiple Imputation by Chained Equations
# 결측치가 포함된 데이터 입력 받아 몬테카를로 시뮬레이션을 반복하여 결측치를 채워넣는 방법 
# pip install impyute
# https://impyute.readthedocs.io/en/master/
import impyute as impy

df2 = pd.DataFrame(impy.mice(df.values))
df2.columns = df.columns
print(df2)

# 모델로 채우기 - IterativeImputer 
# pip install fancyimpute
# https://github.com/iskandr/fancyimpute
from fancyimpute import KNN

df2 = pd.DataFrame(IterativeImputer().fit_transform(df.values))
df2.columns = df.columns
print(df2)

# 모델로 채우기 - KNN
from fancyimpute import KNN

df2 = pd.DataFrame(KNN(k=3).fit_transform(df.values))
df2.columns = df.columns
print(df2)


        one       two     three
0  1.169398 -0.534434 -1.237900
1 -0.072794  0.427945 -0.789082
2 -0.354999 -0.503522       NaN
3 -1.448406 -0.095995       NaN
4       NaN       NaN       NaN
        one       two     three
0  1.169398 -0.534434 -1.237900
1 -0.072794  0.427945 -0.789082
2 -0.354999 -0.503522 -1.013491
3 -1.448406 -0.095995 -1.013491
4 -0.176700 -0.176502 -1.013491
        one       two     three
0  1.169398 -0.534434 -1.237900
1 -0.072794  0.427945 -0.789082
2 -0.354999 -0.503522 -0.999734
3 -1.448406 -0.095995 -0.964769
4 -0.176700 -0.176502 -0.997871
Imputing row 1/5 with 0 missing, elapsed time: 0.000
[KNN] Warning: 3/15 still missing after imputation, replacing with 0
        one       two     three
0  1.169398 -0.534434 -1.237900
1 -0.072794  0.427945 -0.789082
2 -0.354999 -0.503522 -0.919018
3 -1.448406 -0.095995 -0.894652
4  0.000000  0.000000  0.000000


##### Categorical Data

In [137]:
import pandas as pd 
import numpy as np

df = pd.DataFrame(np.random.randint(0, 10, size=(5, 3)), index=range(5), columns=['one', 'two', 'three'])
df = df.astype(str)
df.iat[-1, 0] = np.nan
df.iat[-1, 1] = np.nan
df.iloc[2:, 2] = np.nan
print(df)

# NaN 자체를 하나의 Label 로 다루기
df['one'].fillna(11, inplace=True)
print(df)

# 최빈도 값으로 채우기
mode = df['two'].mode()[0]
print("mode={}".format(mode))
df['two'].fillna(mode, inplace=True)
print(df)

   one  two three
0    2    6     8
1    8    9     7
2    3    3   NaN
3    0    0   NaN
4  NaN  NaN   NaN
  one  two three
0   2    6     8
1   8    9     7
2   3    3   NaN
3   0    0   NaN
4  11  NaN   NaN
mode=0
  one two three
0   2   6     8
1   8   9     7
2   3   3   NaN
3   0   0   NaN
4  11   0   NaN


#### Sequence Data
##### No Trend, No Seasonality
이 경우는 기존의 Record Data 와 같은 데이터형태 이므로 Imputation 방법을 활용하면 됩니다.

##### Trend, No Seasonality
데이터에 경향성은 있지만 주기성은 없는 경우는 interpolate(보간법)을 이용합니다. 

In [142]:
s = pd.Series([0, 1, np.nan, 3])
print(s)
print(s.interpolate())

0    0.0
1    1.0
2    NaN
3    3.0
dtype: float64
0    0.0
1    1.0
2    2.0
3    3.0
dtype: float64


##### Trend, Seasionality

TBD..